### Gathering Rating of User

In [36]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# data = {
#     'Game': ['PUBG', 'ARK Survival', 'The Forest', 'FIFA 23', 'NBA 2K24'],
#     'A': [5, 5, '?', 1, 1],
#     'B': [5, '?', 4, 1, 0],
#     'C': [0, '?', 1, 4, 5],
#     'D': [0, 0, '?', 4, '?'],
#     'E': [1, '?', '?', 4, '?'],
#     'F': ['?', '?', 1, '?', '?']
# }

# Load the CSV file into a DataFrame
df_loaded = pd.read_csv('ratings.csv')


# Pivot the DataFrame to get the original format
df = df_loaded.pivot(index='Game', columns='User', values='Rating').reset_index()


# Ensure the order of games is as specified
game_order = ['PUBG', 'ARK Survival', 'The Forest', 'FIFA 23', 'NBA 2K24']
df.fillna('?', inplace=True)
df['Game'] = pd.Categorical(df['Game'], categories=game_order, ordered=True)
df.sort_values('Game', inplace=True)
df = df.reset_index()
del df['index']



C:\Users\Admin\AppData\Local\Temp\ipykernel_19116\699501721.py:24: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '?' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.fillna('?', inplace=True)


In [37]:
df

User,Game,A,B,C,D,E,F
0,PUBG,5.0,5.0,0.0,0.0,1.0,?
1,ARK Survival,5.0,?,?,0.0,?,?
2,The Forest,?,4.0,1.0,?,?,1.0
3,FIFA 23,1.0,1.0,4.0,4.0,4.0,?
4,NBA 2K24,1.0,0.0,5.0,?,?,?


### Step 2 Build Profile 

In [38]:
df.replace('?', 0, inplace=True)


C:\Users\Admin\AppData\Local\Temp\ipykernel_19116\1796858411.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('?', 0, inplace=True)


In [39]:
df

User,Game,A,B,C,D,E,F
0,PUBG,5.0,5.0,0.0,0.0,1.0,0.0
1,ARK Survival,5.0,0.0,0.0,0.0,0.0,0.0
2,The Forest,0.0,4.0,1.0,0.0,0.0,1.0
3,FIFA 23,1.0,1.0,4.0,4.0,4.0,0.0
4,NBA 2K24,1.0,0.0,5.0,0.0,0.0,0.0


### Step 3: Calculate Similarity

In [40]:
# Tính toán ma trận cosine similarity
features = df.drop('Game', axis=1)
cosine_sim = cosine_similarity(features)

In [41]:
cosine_sim

array([[1.        , 0.70014004, 0.66009836, 0.27724131, 0.13730876],
       [0.70014004, 1.        , 0.        , 0.14142136, 0.19611614],
       [0.66009836, 0.        , 1.        , 0.26666667, 0.23112508],
       [0.27724131, 0.14142136, 0.26666667, 1.        , 0.58243521],
       [0.13730876, 0.19611614, 0.23112508, 0.58243521, 1.        ]])

### Step: 4 Build Recommend Funcion

In [42]:
# Tạo hàm gợi ý dựa trên tên game
def recommend(game, cosine_sim=cosine_sim):
    idx = df[df['Game'] == game].index[0]
    sim_scores = list(enumerate(cosine_sim[idx]))
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    # sim_scores = sim_scores[1:4]  # Lấy 3 game tương tự nhất
    game_indices = [i[0] for i in sim_scores]
    return df['Game'].iloc[game_indices]

### Step 5: Testing Recommendation

In [43]:
# Ví dụ: Gợi ý game tương tự PUBG
print(recommend('FIFA 23'))

3         FIFA 23
4        NBA 2K24
0            PUBG
2      The Forest
1    ARK Survival
Name: Game, dtype: category
Categories (5, object): ['PUBG' < 'ARK Survival' < 'The Forest' < 'FIFA 23' < 'NBA 2K24']
